## Data processing

This Jupyter Noterbook helps us to convert binary attribute(s) to +/-1, categorical attributes(s) to onehot.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

We load the data which were cleaned from the `data cleaning` step.

In [2]:
Xy = np.loadtxt('tanzania_cleaned.dat', dtype = 'str')

print(Xy.shape)
print(Xy)

(671, 20)
[['42.0' '1.0' '2.0' ... '6.62987012987013' '6.0' '1.0']
 ['40.0' '9.0' '1.0' ... '6.62987012987013' '6.0' '1.0']
 ['38.0' '1.0' '1.0' ... '6.62987012987013' '6.0' '1.0']
 ...
 ['40.0' '1.0' '2.0' ... '6.62987012987013' '3.088709677419355' '0.0']
 ['40.0' '9.0' '1.0' ... '6.62987012987013' '3.088709677419355' '0.0']
 ['36.335559265442406' '9.0' '1.0' ... '6.62987012987013'
  '3.088709677419355' '0.0']]


### Attributes

We find number of unique value for each column, to have an idea about which variables are continuous, which variables are binary, category. It depends on data, however it is likely that nu = 2 --> binary; nu = 3 or 4: --> category, n > 4: continuous. Of course, we have to see data in detail as well.

In [3]:
X = Xy[:,:-1]
l,n = X.shape
nu = np.array([len(np.unique(X[:,i])) for i in range(n)])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[ 17   4   4   5   3   7 302   9  12   2   2   3   4   4  52 100  74  20
   6]


In [4]:
i_binary = []
i_category = []
i_continuous = []
for i in range(X.shape[1]):
    nu = np.unique(X[:,i])
    if len(nu) == 2: # binary 
        i_binary.append(i)
    elif len(nu) < 5:
        i_category.append(i)
    else:
        i_continuous.append(i)
        
print('i_binary:',i_binary)
print('i_category:',i_category)

i_binary: [9, 10]
i_category: [1, 2, 4, 11, 12, 13]


We then define variable type, 1: continuous, 2: binary, 3: category.

In [5]:
variable_type  = np.ones(n) # continuous
variable_type[i_binary] = 2 # binary
variable_type[i_category] = 3 # categorical

print(variable_type)

[1. 3. 3. 1. 3. 1. 1. 1. 1. 2. 2. 3. 3. 3. 1. 1. 1. 1. 1.]


We now convert binary to +/-1, category to onehot.

In [6]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # continuous
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))
        elif i_type == 2: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))
        else: # category      
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))        

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [7]:
# convert X
X_new = convert_binary_and_category(X,variable_type)

print(X_new.shape)
print(X_new)

(671, 35)
[[42.          1.          0.         ... 60.          6.62987013
   6.        ]
 [40.          0.          0.         ... 50.          6.62987013
   6.        ]
 [38.          1.          0.         ... 71.          6.62987013
   6.        ]
 ...
 [40.          1.          0.         ... 78.35309973  6.62987013
   3.08870968]
 [40.          0.          0.         ... 78.35309973  6.62987013
   3.08870968]
 [36.33555927  0.          0.         ... 92.          6.62987013
   3.08870968]]


### Target

In [8]:
## target
y = Xy[:,-1].astype(float)

print(np.unique(y,return_counts=True))

(array([0., 1.]), array([547, 124]))


In [9]:
y_new = y
# convert taget to 0 and 1
#y_new = np.ones(y.shape[0])


print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([547, 124]))


In [10]:
# combine X and y
Xy_new = np.hstack((X_new,y_new[:,np.newaxis]))

In [11]:
np.savetxt('tanzania_processed.dat',Xy_new,fmt='%f')